In [1]:
from models.classifier import TransformerEncoder
from pytorch_lightning import Trainer
import torch

In [2]:
hparams = {
    "hidden_size": 48,  # size of the hidden layers and embeddings
    "hidden_ff": 96,  # size of the position-wise feed-forward layer
    "n_encoders": 4,  # number of encoder blocks
    "n_heads": 2,  # number of attention heads in the multiheadattention module
    "n_local": 2,  # number of local attention heads
    "local_window_size": 4,  # size of the window for local attention
    'batch_size': 4,
    "max_length": 30,  # maximum length of the input sequence
    "vocab_size": 100,  # size of the vocabulary
    "learning_rate": 0.001,
    "num_epochs": 30,
    "attention_type": "performer",
    "norm_type": "rezero",
    "num_random_features": 32,  # number of random features for the Attention module (Performer uses this)
    "emb_dropout": 0.1,  # dropout for the embedding block
    "fw_dropout": 0.1,  # dropout for the position-wise feed-forward layer
    "att_dropout": 0.1,  # dropout for the multiheadattention module
    "dc_dropout": 0.1,  # dropout for the decoder block
    "hidden_act": "swish",  # activation function for the hidden layers (attention layers use ReLU)
    "epsilon": 1e-8,
    "weight_decay": 0.01,
    "beta1": 0.9,
    "beta2": 0.999,
}
model = TransformerEncoder(hparams)

#### Sanity Check

In [12]:
batch = {
    'tokens': torch.randint(0, model.hparams.vocab_size, (model.hparams.batch_size, model.hparams.max_length)),
    'abspos': torch.arange(0, model.hparams.max_length).unsqueeze(0).repeat(model.hparams.batch_size, 1),
    'age': torch.randint(0, 100, (model.hparams.batch_size, 1)).repeat(1, model.hparams.max_length),
    'padding_mask': torch.zeros(model.hparams.batch_size, model.hparams.max_length),
    'targets': torch.randint(0, 2, (model.hparams.batch_size, 1)).float(),
}

In [13]:
batch

{'tokens': tensor([[21, 89, 21, 15, 22, 10,  4, 37, 32, 53, 88, 55, 46, 37, 51, 25,  2,  4,
          90, 74, 82, 71, 11, 12,  8, 59, 13, 89,  8, 67],
         [60, 11, 72, 46, 21, 55, 16, 91,  1, 99,  0, 17, 69, 72, 88, 29, 61, 93,
          45, 31, 40, 92,  1, 49,  9,  4, 41, 28, 32, 68],
         [43, 78,  6, 74, 15, 47, 25,  6, 95, 38, 54, 36,  8, 82, 25, 68, 62, 49,
          18, 94, 41, 38, 26, 54, 78, 79, 53,  6, 83, 56],
         [29, 41, 47, 77, 64, 61, 35, 20,  0, 13,  8, 57, 98, 68, 32, 45, 30, 87,
          28, 57,  6, 75, 51, 92, 41, 86, 53, 15, 49,  4]]),
 'abspos': tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 

In [14]:
model(batch)

{'logits': tensor([[-0.0677],
         [-0.0677],
         [-0.0677],
         [-0.0677]], grad_fn=<AddmmBackward0>),
 'preds': tensor([[0.4831],
         [0.4831],
         [0.4831],
         [0.4831]], grad_fn=<SigmoidBackward0>)}

In [15]:
model.training_step(batch, 0)

tensor(0.7106, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [16]:
model.on_train_epoch_end()


Train Metrics
	Loss: 0.694
	Accuracy: 0.501
	MCC: -0.002



### Full pipeline would be something like

In [17]:
from dataloaders.synthetic import SyntheticDataModule
dataloader = SyntheticDataModule(num_samples=1000, max_length=hparams['max_length'],
                                  batch_size=hparams['batch_size'], vocab_size=hparams['vocab_size'])

In [18]:
trainer = Trainer(max_epochs=30,
                accelerator="cpu",   ### change to "cuda" or "gpu" or 'msp'
                limit_train_batches=0.5,
                accumulate_grad_batches=4,
                num_sanity_val_steps=8,
                check_val_every_n_epoch=1)
trainer.fit(model, dataloader)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/carlomarx/.local/share/virtualenvs/life2vec-light-Ez8u7ZRp-python/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Missing logger folder: /Users/carlomarx/Documents/GitHub/project2vec/lightning_logs

   | Name        | Type                   | Params | Mode
---------------------------------------------------------------
0  | transformer | Transformer            | 79.5 K | eval
1  | decoder     | CLS_Decoder            | 2.4 K  | eval
2  | loss        | BCEWithLogitsLoss      | 0      | eval
3  | train_loss  | MeanMetric             | 0      | eval
4  | valid_loss  | MeanMetric             | 0      | eval
5  | test_loss   | MeanMetric             | 0      | eval
6  | train_acc   | BinaryAccuracy         | 0      | eval
7  | valid_acc   | BinaryAccuracy         | 0  

/Users/carlomarx/.local/share/virtualenvs/life2vec-light-Ez8u7ZRp-python/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/carlomarx/.local/share/virtualenvs/life2vec-light-Ez8u7ZRp-python/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 125/125 [00:01<00:00, 76.47it/s, v_num=0]

/Users/carlomarx/.local/share/virtualenvs/life2vec-light-Ez8u7ZRp-python/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
